In [1]:
import LemurBounce2
l = LemurBounce2.LemurBounce2()

In [70]:
l.sendOSCMessage('/loadWorld', 1, '1')

/collision
[2.0, 0.0, 'start']
/collision
[2.0, 0.0, 'end']
/collision
[8.0, 0.0, 'start']
/collision
[8.0, 0.0, 'end']
/collision
[0.0, 3.0, 'start']
/collision
[5.0, 3.0, 'start']
/collision
[0.0, 3.0, 'end']
/collision
[5.0, 3.0, 'end']
/collision
[7.0, 2.0, 'start']
/collision
[7.0, 2.0, 'end']
/collision
[0.0, 3.0, 'start']
/collision
[8.0, 0.0, 'start']
/collision
[8.0, 0.0, 'end']
/collision
[0.0, 3.0, 'end']
/collision
[0.0, 3.0, 'start']
/collision
[0.0, 3.0, 'end']
/collision
[8.0, 0.0, 'start']
/collision
[9.0, 2.0, 'start']
/collision
[1.0, 0.0, 'start']
/collision
[8.0, 0.0, 'end']
/collision
[9.0, 2.0, 'end']
/collision
[1.0, 0.0, 'end']
/collision
[5.0, 2.0, 'start']
/collision
[3.0, 1.0, 'start']
/collision
[5.0, 2.0, 'end']
/collision
[3.0, 1.0, 'end']
/collision
[7.0, 1.0, 'start']
/collision
[7.0, 1.0, 'end']
/collision
[7.0, 2.0, 'start']
/collision
[7.0, 2.0, 'end']
/collision
[0.0, 2.0, 'start']
/collision
[0.0, 2.0, 'end']
/collision
[0.0, 2.0, 'start']
/collisio

/collision
[0.0, 0.0, 'start']
/collision
[0.0, 0.0, 'end']
/collision
[2.0, 0.0, 'start']
/collision
[2.0, 0.0, 'end']
/collision
[9.0, 1.0, 'start']
/collision
[9.0, 1.0, 'end']
/collision
[1.0, 0.0, 'start']
/collision
[1.0, 0.0, 'end']
/collision
[2.0, 0.0, 'start']
/collision
[2.0, 0.0, 'end']
/collision
[0.0, 3.0, 'start']
/collision
[0.0, 3.0, 'end']
/collision
[4.0, 1.0, 'start']
/collision
[4.0, 1.0, 'end']
/collision
[4.0, 2.0, 'start']
/collision
[4.0, 2.0, 'end']
/collision
[2.0, 1.0, 'start']
/collision
[2.0, 1.0, 'end']
/collision
[9.0, 1.0, 'start']
/collision
[9.0, 1.0, 'end']
/collision
[7.0, 0.0, 'start']
/collision
[7.0, 0.0, 'end']
/collision
[1.0, 2.0, 'start']
/collision
[1.0, 2.0, 'end']
/collision
[8.0, 3.0, 'end']
/collision
[6.0, 3.0, 'start']
/collision
[6.0, 3.0, 'end']
/collision
[1.0, 2.0, 'start']
/collision
[1.0, 2.0, 'end']
/collision
[9.0, 2.0, 'start']
/collision
[9.0, 2.0, 'end']
/collision
[9.0, 2.0, 'start']
/collision
[1.0, 1.0, 'start']
/collisio

/collision
[5.0, 0.0, 'start']
/collision
[5.0, 0.0, 'end']
/collision
[6.0, 2.0, 'end']
/collision
[0.0, 2.0, 'end']
/collision
[1.0, 2.0, 'start']
/collision
[4.0, 3.0, 'start']
/collision
[4.0, 3.0, 'end']
/collision
[4.0, 0.0, 'start']
/collision
[4.0, 0.0, 'end']
/collision
[4.0, 1.0, 'start']
/collision
[4.0, 1.0, 'end']
/collision
[1.0, 2.0, 'end']
/collision
[1.0, 3.0, 'start']
/collision
[1.0, 3.0, 'end']
/collision
[4.0, 3.0, 'start']
/collision
[4.0, 3.0, 'end']
/collision
[4.0, 0.0, 'start']
